In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import PReLU
from tensorflow.keras.models import Sequential

In [2]:
expressions = ['happy', 'sad', 'fear', 'neutral', 'angry', 'suprise', 'disgust']
x_train, y_train, x_test, y_test = [], [], [], []

In [3]:
file_path = '/project/train/scr_repo1/data6/'
for file in os.listdir(file_path):
    img_file = os.path.join(file_path,file)
    for img in  os.listdir(img_file):
        if img.endswith('.jpg'):
            for i in range(len(expressions)):
                if(expressions[i]==file):
                    print(os.path.join(img_file,img))
                    img = load_img(os.path.join(img_file,img),color_mode="grayscale")
                    img = img_to_array(img)
                    print(img)
                    x_train.append(img)
                    y_train.append(i)
x_train = np.array(x_train)/255.0
y_train = np.array(y_train).astype('int') 
y_train = to_categorical(y_train, num_classes = 7)

/project/train/scr_repo1/data6/happy/0f9fde7faa8146639ffb3ff3f2767fd5.jpg
[[[65.]
  [57.]
  [50.]
  ...
  [23.]
  [27.]
  [28.]]

 [[62.]
  [55.]
  [48.]
  ...
  [23.]
  [26.]
  [28.]]

 [[53.]
  [51.]
  [51.]
  ...
  [22.]
  [26.]
  [27.]]

 ...

 [[55.]
  [62.]
  [71.]
  ...
  [70.]
  [69.]
  [69.]]

 [[58.]
  [65.]
  [76.]
  ...
  [71.]
  [70.]
  [70.]]

 [[59.]
  [66.]
  [78.]
  ...
  [71.]
  [70.]
  [70.]]]
/project/train/scr_repo1/data6/happy/1a6dde0d9d454770a339ec48d8117232.jpg
[[[ 0.]
  [13.]
  [ 0.]
  ...
  [ 6.]
  [43.]
  [64.]]

 [[ 5.]
  [14.]
  [12.]
  ...
  [ 1.]
  [ 9.]
  [18.]]

 [[ 4.]
  [ 3.]
  [14.]
  ...
  [ 8.]
  [ 5.]
  [ 0.]]

 ...

 [[41.]
  [24.]
  [ 9.]
  ...
  [59.]
  [70.]
  [92.]]

 [[45.]
  [27.]
  [ 6.]
  ...
  [26.]
  [22.]
  [74.]]

 [[32.]
  [25.]
  [ 0.]
  ...
  [41.]
  [26.]
  [45.]]]
/project/train/scr_repo1/data6/happy/0fcac4c8df11442d873c4290a60c2a48.jpg
[[[130.]
  [143.]
  [ 88.]
  ...
  [ 32.]
  [ 36.]
  [ 46.]]

 [[108.]
  [125.]
  [ 69.]
  ...

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=2021)

In [5]:
x_train

array([[[[0.29411766],
         [0.30588236],
         [0.29803923],
         ...,
         [0.1254902 ],
         [0.13333334],
         [0.15686275]],

        [[0.2901961 ],
         [0.30588236],
         [0.29803923],
         ...,
         [0.13333334],
         [0.13725491],
         [0.15294118]],

        [[0.28627452],
         [0.3019608 ],
         [0.29803923],
         ...,
         [0.14901961],
         [0.14901961],
         [0.14901961]],

        ...,

        [[0.3254902 ],
         [0.31764707],
         [0.3137255 ],
         ...,
         [0.3882353 ],
         [0.4745098 ],
         [0.5294118 ]],

        [[0.3254902 ],
         [0.32156864],
         [0.3137255 ],
         ...,
         [0.4509804 ],
         [0.50980395],
         [0.54509807]],

        [[0.3254902 ],
         [0.32156864],
         [0.31764707],
         ...,
         [0.44705883],
         [0.49019608],
         [0.5411765 ]]],


       [[[0.42352942],
         [0.3137255 ],
         [0.28

In [6]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), strides=1, padding='same', activation='relu',input_shape=(256,256,1)))
model.add(Conv2D(32, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5)) 

# model.add(Conv2D(64, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
# model.add(Conv2D(64, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.5)) 

# model.add(Conv2D(128, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
# model.add(Conv2D(128, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5)) 

model.add(Flatten())
model.add(Dense(1024, activation='relu'))   
model.add(Dropout(0.5)) 
model.add(Dense(128, activation='relu')) 
model.add(Dense(7, activation='softmax')) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# early_stopping =EarlyStopping(monitor='acc', patience=10, verbose=2)

In [8]:
history = model.fit(x_train,y_train,epochs=200,batch_size=16)

ValueError: Error when checking input: expected conv2d_input to have shape (256, 256, 1) but got array with shape (48, 48, 1)

In [ ]:
score_train = model.evaluate(x_train, y_train)
print('Train accuracy:', score_train)
score_test = model.evaluate(x_test,y_test)
print('Train accuracy:', score_test)